In [2]:
import os

# Установите переменную окружения JAVA_HOME
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']

# Проверьте версию Java
!java -version

openjdk version "1.8.0_422"
OpenJDK Runtime Environment (build 1.8.0_422-8u422-b05-1~24.04-b05)
OpenJDK 64-Bit Server VM (build 25.422-b05, mixed mode)


In [3]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName('less_pysqprk_DF').\
        master('local').\
        config('spark.executor.memory', "512m").\
        config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000").\
        config("spark.hadoop.fs.default.name", "hdfs://localhost:9000").\
        getOrCreate()

/home/ilyagasan/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [4]:
from pyspark.sql.functions import udf

from pyspark.sql.types import StringType, FloatType

df = spark.read.csv('hdfs://localhost:9000/data/Sales_April_2019.csv', header=True, inferSchema=True)

In [8]:
from pyspark.sql.functions import col

In [5]:
df.show()

+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  176558|USB-C Charging Cable|               2|     11.95|04/19/19 08:46|917 1st St, Dalla...|
|    null|                null|            null|      null|          null|                null|
|  176559|Bose SoundSport H...|               1|     99.99|04/07/19 22:30|682 Chestnut St, ...|
|  176560|        Google Phone|               1|     600.0|04/12/19 14:38|669 Spruce St, Lo...|
|  176560|    Wired Headphones|               1|     11.99|04/12/19 14:38|669 Spruce St, Lo...|
|  176561|    Wired Headphones|               1|     11.99|04/30/19 09:27|333 8th St, Los A...|
|  176562|USB-C Charging Cable|               1|     11.95|04/29/19 13:03|381 Wilson St, Sa...|
|  176563|Bose SoundSport H...|         

In [6]:
def my_udf(column):
    return  str(column).upper()

my_udf('oaky')

'OAKY'

In [7]:
my_udf = udf(my_udf, StringType())

In [9]:
df = df.withColumn('upper_Product', my_udf(col("Product")))

In [10]:
df.show()

+--------+--------------------+----------------+----------+--------------+--------------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|       upper_Product|
+--------+--------------------+----------------+----------+--------------+--------------------+--------------------+
|  176558|USB-C Charging Cable|               2|     11.95|04/19/19 08:46|917 1st St, Dalla...|USB-C CHARGING CABLE|
|    null|                null|            null|      null|          null|                null|                NONE|
|  176559|Bose SoundSport H...|               1|     99.99|04/07/19 22:30|682 Chestnut St, ...|BOSE SOUNDSPORT H...|
|  176560|        Google Phone|               1|     600.0|04/12/19 14:38|669 Spruce St, Lo...|        GOOGLE PHONE|
|  176560|    Wired Headphones|               1|     11.99|04/12/19 14:38|669 Spruce St, Lo...|    WIRED HEADPHONES|
|  176561|    Wired Headphones|               1|     11.99|04/30